In [1]:
import torch as tr
import su2_chain as s
import integrators as integ

In [2]:
a = tr.normal(0.0,2.0,[4])
a = 2*a/tr.norm(a)
print(a)
U = s.Mat(a) #tr.sum(a[:,None,None] * basis,dim=0)
print(U)
print("Should be identity")
print(s.TdxT(U,U))

tensor([-0.5343, -1.1274, -0.7276, -1.3835])
tensor([[-0.2671-0.6918j,  0.3638-0.5637j],
        [-0.3638-0.5637j, -0.2671+0.6918j]])
Should be identity
tensor([[1.0000-7.5865e-09j, 0.0000-2.9802e-08j],
        [0.0000-2.9802e-08j, 1.0000+7.5865e-09j]])


In [3]:
b = tr.normal(0.0,2.0,[4])
b = 2*b/tr.norm(b)
print(b)
V = s.Mat(b) #tr.sum(a[:,None,None] * basis,dim=0)
print(V)
print("Should be identity")
print(s.TdxT(V,V))

tensor([ 0.9128,  0.7590,  0.3354, -1.5742])
tensor([[ 0.4564-0.7871j, -0.1677+0.3795j],
        [ 0.1677+0.3795j,  0.4564+0.7871j]])
Should be identity
tensor([[1.+1.9956e-09j, 0.-7.4506e-09j],
        [0.-7.4506e-09j, 1.-1.9956e-09j]])


In [4]:
tr.trace(U@tr.conj(V.T)).real.item()/2-(a@b/4).item()

1.4901161193847656e-08

In [5]:
su2f = s.field([4,4],Nbatch=2)

In [6]:
su2c = s.SU2chain(beta=3.0,field_type=su2f)

In [7]:
P = su2c.refreshP()

In [8]:
U =su2c.f.hot()
U.shape

torch.Size([2, 4, 4, 4])


torch.Size([2, 4, 4, 2, 2])

In [9]:
a= su2c.f.coeffs(U)

In [10]:
a.shape

torch.Size([2, 4, 4, 4])

In [11]:
s.Mat(a[0,0,0,:])- U[0,0,0,:,:]

tensor([[0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]])

In [12]:
FF = su2c.f.mat(a)

In [13]:
(FF-U).norm()

tensor(0.)

In [14]:
P.shape

torch.Size([2, 4, 4, 2, 2])

In [15]:
P[...,1,0].real.mean(), P[...,1,0].real.std(),P[...,1,0].imag.mean(), P[...,1,0].imag.std()

(tensor(0.0346), tensor(0.5182), tensor(-0.1081), tensor(0.4859))

In [16]:
P[...,0,0].real.mean(), P[...,0,0].real.std(),P[...,0,0].imag.mean(), P[...,0,0].imag.std()

(tensor(0.), tensor(0.), tensor(0.1283), tensor(0.4950))

In [17]:
tr.sqrt(tr.tensor(2))

tensor(1.4142)

In [18]:
pp = 2.0*tr.einsum('kij,...ji->...k',s.tau,P)

In [19]:
pp.real.std(dim=(0,1,2))

tensor([0.9719, 1.0364, 0.9899])

In [20]:
pp.shape

torch.Size([2, 4, 4, 3])

In [21]:
ff = -tr.einsum('...k,kij->...ij',pp,s.tau)

In [22]:
ff[0,0,0]-P[0,0,0]

tensor([[-0.-1.4901e-08j, 0.+0.0000e+00j],
        [0.+0.0000e+00j, 0.-1.4901e-08j]])

In [23]:
tr.trace(P[0,0,0])

tensor(0.+2.9802e-08j)

In [24]:
tr.einsum('...ij,...ij->',P,P.conj()).real

tensor(48.3551)

In [25]:
tr.einsum('...k,...k->',pp,pp)/2

tensor(48.3551+0.j)

In [26]:
(tr.norm(P,dim=(su2c.f.ci[0],su2c.f.ci[1])) - tr.norm(pp,dim=(len(pp.shape)-1))/tr.sqrt(tr.tensor(2))).norm()

tensor(3.7253e-07)

In [27]:
E = su2c.f.expo(P)

In [28]:
E[0,0,0] @ E[0,0,0].conj().transpose(0,1)

tensor([[1.0000e+00+0.0000e+00j, 1.0272e-08+9.3132e-09j],
        [1.0272e-08-9.3132e-09j, 1.0000e+00+0.0000e+00j]])

In [29]:
tr.eye(2,2) + P[0,0,0]@(tr.eye(2,2) + 0.5*P[0,0,0]@(tr.eye(2,2)+ 1/3*P[0,0,0]@(tr.eye(2,2) + 1/4*P[0,0,0]))), E[0,0,0]

(tensor([[ 0.9175+0.1953j, -0.3344-0.0904j],
         [ 0.3344-0.0904j,  0.9175-0.1953j]]),
 tensor([[ 0.9175+0.1954j, -0.3345-0.0905j],
         [ 0.3345-0.0905j,  0.9175-0.1954j]]))

In [30]:
P = su2c.refreshP()
Ken = su2c.kinetic(P)
Ven = su2c.action(U)
Hi = Ken + Ven
print("The total initial kinetic energy is: ",Ken)
print("The total initial potential energy is: ",Ven)
print("The total initial energy is: ",Hi)

The total initial kinetic energy is:  tensor([22.6581, 22.3999])
The total initial potential energy is:  tensor([ 97.8376, 104.5572])
The total initial energy is:  tensor([120.4957, 126.9571])


In [31]:
dt = 0.1
k = 10
l = integ.leapfrog(su2c.force,su2f.evolveQ,k,1.0)
m = integ.minnorm2(su2c.force,su2f.evolveQ,k,1.0)
PP ,QQ  = l.integrate(P,U)
PP2,QQ2 = m.integrate(P,U)

In [32]:
KenF = su2c.kinetic(PP)
VenF = su2c.action(QQ)
Hf = KenF + VenF
print("The total initial kinetic energy is: ",KenF)
print("The total initial potential energy is: ",VenF)
print("The total initial energy is: ",Hf)
print("The total energy diff is: ",Hf-Hi)

The total initial kinetic energy is:  tensor([70.1550, 52.5747])
The total initial potential energy is:  tensor([50.0433, 74.2198])
The total initial energy is:  tensor([120.1983, 126.7945])
The total energy diff is:  tensor([-0.2974, -0.1626])


In [33]:
KenF = su2c.kinetic(PP2)
VenF = su2c.action(QQ2)
Hf = KenF + VenF
print("The total initial kinetic energy is: ",KenF)
print("The total initial potential energy is: ",VenF)
print("The total initial energy is: ",Hf)
print("The total energy diff is: ",Hf-Hi)

The total initial kinetic energy is:  tensor([70.5167, 52.8252])
The total initial potential energy is:  tensor([49.9594, 74.1227])
The total initial energy is:  tensor([120.4761, 126.9479])
The total energy diff is:  tensor([-0.0196, -0.0092])


In [34]:
foo = tr.einsum('kij,klm->ijlm',s.tau,s.tau)

In [35]:
foo[0,0,1,1]

tensor(0.2500+0.j)

In [36]:
2.0*tr.trace(s.tau[0]@s.tau[0].T.conj())

tensor(1.+0.j)

In [37]:
Pa=tr.tensor([2.0*tr.trace(P[0,0,0]@s.tau[0].T.conj()),2.0*tr.trace(P[0,0,0]@s.tau[1].T.conj()),2.0*tr.trace(P[0,0,0]@s.tau[2].T.conj())])

In [38]:
tr.trace(P[0,0,0]@P[0,0,0].T.conj())

tensor(1.5806+0.j)

In [39]:
(Pa*Pa).sum()/2

tensor(1.5806+0.j)

In [40]:
P[0,0,0]

tensor([[ 0.0000+0.4797j, -0.6143-0.4276j],
        [ 0.6143-0.4276j,  0.0000-0.4797j]])

In [41]:
tr.norm(tr.einsum('aij,a->ij',s.tau,Pa) - P[0,0,0])

tensor(0.)

In [42]:
su2c.kinetic(P)

tensor([22.6581, 22.3999])

In [43]:
# the kinnetic energy derivative
eps = 10.0
D = tr.zeros_like(P)

for x in range(P.shape[1]):
    for y in range(P.shape[2]):
        for a in range(3):
            PPp = P.clone()
            PPm = P.clone()
            #PPp[:,x,y] = tr.einsum('ik,bkj->bij',Rp[a],PPp[:,x,y])
            #PPm[:,x,y] = tr.einsum('ik,bkj->bij',Rm[a],PPm[:,x,y])
            PPp[:,x,y] +=  eps*s.tau[a]
            PPm[:,x,y] -=  eps*s.tau[a]
            foo = (su2c.kinetic(PPp) - su2c.kinetic(PPm))/2.0/eps
            for b in range(D.shape[0]):
                D[b,x,y] += s.tau[a]*foo[b]
            
            

In [44]:
tr.norm(D-P)/tr.norm(P)

tensor(7.8333e-07)

In [45]:
# the potential energy derivative 
eps = 0.01
Rp = tr.zeros([3,2,2],dtype=tr.complex64)
Rm = tr.zeros([3,2,2],dtype=tr.complex64)
Dact = tr.zeros_like(P)
for k in range(3):
    Rp[k] = su2c.f.expo(+eps*s.tau[k].unsqueeze(0).unsqueeze(0).unsqueeze(0)).squeeze()
    Rm[k] = su2c.f.expo(-eps*s.tau[k].unsqueeze(0).unsqueeze(0).unsqueeze(0)).squeeze()
    

for x in range(P.shape[1]):
    for y in range(P.shape[2]):
        for a in range(3):
            Up = U.clone()
            Um = U.clone()
            Up[:,x,y] = tr.einsum('ik,bkj->bij',Rp[a],Up[:,x,y])
            Um[:,x,y] = tr.einsum('ik,bkj->bij',Rm[a],Um[:,x,y])
            foo = (su2c.action(Up) - su2c.action(Um))/2.0/eps
            for b in range(D.shape[0]):
                Dact[b,x,y] += s.tau[a]*foo[b]



In [46]:
Dact[0,0,0]

tensor([[ 0.0000-0.3569j, -0.6090-0.7807j],
        [ 0.6090-0.7807j,  0.0000+0.3569j]])

In [47]:
F=su2c.force(U)

In [48]:
tr.norm(F+Dact)/tr.norm(F)

tensor(0.0002)